# P/P/1 and U/U/1 Queue Size Evaluation

In class, Prof. Moshe said there were two methods for finding the queue size of G/G/1 models with the **arrival process that does not follow a Poisson process**. They are: (Slides P271)

1. Use a separate independent Poisson inspector.

2. Consider both the queue-size measurements and the time they remain at that level.

This notebook calculates the mean queue size of P/P/1 and U/U/1 using method 1.

In [2]:
from scipy.stats import pareto
from scipy.stats import expon
import numpy as np

Using the Poisson inspector, be aware that we should calculate **the queue size at the time of the inspector** instead of the queue size **on arrival**.

## P/P/1 Simulation
For the **P/P/1** simulation, the parameters of the **inter-arrival time** are $\alpha=3$ and $x_{\min}=1$, and the parameters of the **service time** are $\alpha=5$ and $x_{\min}=1$. Theoretically, the arrival rate should be $\frac{1}{MeanInterArrivalTime}=\frac{1}{\frac{x_{\min}\times\alpha}{\alpha-1}}=\frac{2}{3}$.

In [3]:
arr_alpha = 3  # arrival alpha
arr_x_min = 1  # arrival minimum x
service_alpha = 5  # service alpha
service_x_min = 1  # service minimum x
total_arrivals = 10000

arrival_time = np.zeros(total_arrivals) # the time when the customer arrives 
service_time = np.zeros(total_arrivals) # the service time
enter_time = np.zeros(total_arrivals)   # the time when the customer is served
departure_time = np.zeros(total_arrivals)  # the time when the customer leaves the queue

# initialization of service time and arrival time
for i in range(total_arrivals):
    service_time[i] = pareto.rvs(service_alpha, scale = service_x_min)
for i in range(1, total_arrivals):
    inter_arrival_time = pareto.rvs(arr_alpha, scale = arr_x_min)
    arrival_time[i] = arrival_time[i - 1] + inter_arrival_time
    
departure_time[0] = service_time[0]

# simulation process
for i in range(1, total_arrivals):
    if departure_time[i - 1] < arrival_time[i]:
        enter_time[i] = arrival_time[i]
    else:
        enter_time[i] = departure_time[i - 1]
    departure_time[i] = enter_time[i] + service_time[i]

In [4]:
rate = 1 # lambda
T = departure_time[total_arrivals - 1]
current_time = 0
selectedMoments = []
while current_time < T:
    point = expon.rvs(scale = 1 / rate)
    current_time += point
    selectedMoments.append(current_time)

delay = np.zeros(len(selectedMoments))
queue_size = np.zeros(len(selectedMoments))
for i in range(len(selectedMoments)):
    idx_departure = np.searchsorted(departure_time, selectedMoments[i]) - 1
    idx_arrival = np.searchsorted(arrival_time, selectedMoments[i]) - 1
    delay[i] = departure_time[idx_arrival] - arrival_time[idx_arrival]
    queue_size[i] = idx_arrival - idx_departure
delay2 = departure_time - arrival_time
mean_queue_size = np.sum(delay2) / departure_time[total_arrivals - 1]
print("E[Delay] =", np.mean(delay))
print("E[Delay2] =", np.mean(delay2))
print("E[Q] =", np.mean(queue_size))

E[Delay] = 1.7281327678042582
E[Delay2] = 1.7168634279660895
E[Q] = 1.135009122237989


Check the result using Little's formula:

$1.7168*\frac{2}{3}=1.1145$

## U/U/1 Simulation

In [5]:
total_arrivals = 10000

arrival_time = np.zeros(total_arrivals) # the time when the customer arrives 
service_time = np.zeros(total_arrivals) # the service time
enter_time = np.zeros(total_arrivals)   # the time when the customer is served
departure_time = np.zeros(total_arrivals)  # the time when the customer leaves the queue

# initialization of service time and arrival time
for i in range(total_arrivals):
    service_time[i] = np.random.uniform(0, 1)
for i in range(1, total_arrivals):
    inter_arrival_time = np.random.uniform(0, 2)
    arrival_time[i] = arrival_time[i - 1] + inter_arrival_time
    
departure_time[0] = service_time[0]

# simulation process
for i in range(1, total_arrivals):
    if departure_time[i - 1] < arrival_time[i]:
        enter_time[i] = arrival_time[i]
    else:
        enter_time[i] = departure_time[i - 1]
    departure_time[i] = enter_time[i] + service_time[i]

In [6]:
rate = 1 # lambda
T = departure_time[total_arrivals - 1]
current_time = 0
selectedMoments = []
while current_time < T:
    point = expon.rvs(scale = 1 / rate)
    current_time += point
    selectedMoments.append(current_time)

delay = np.zeros(len(selectedMoments))
queue_size = np.zeros(len(selectedMoments))
for i in range(len(selectedMoments)):
    idx_departure = np.searchsorted(departure_time, selectedMoments[i]) - 1
    idx_arrival = np.searchsorted(arrival_time, selectedMoments[i]) - 1
    delay[i] = departure_time[idx_arrival] - arrival_time[idx_arrival]
    queue_size[i] = idx_arrival - idx_departure
    
delay2 = departure_time - arrival_time
print("E[Delay] =", np.mean(delay))
print("E[Delay2] =", np.mean(delay2))
print("E[Q] =", np.mean(queue_size))

E[Delay] = 0.6426515341547101
E[Delay2] = 0.6404755813597428
E[Q] = 0.6431201550387597


## Acknowledge
Many thanks to Ms. Linjun DAI for helping me debug.